## Imports


In [1]:
import pandas as pd
import re
import numpy as np
import itertools

## Notes
Copy the extracted pose file to the data and input it here.

In [2]:
df = pd.read_csv("./pose_train.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,296.361400,293.094178,397.665245,206.603340,377.376373,432.355077,305.081375,468.451430,...,205.570562,342.267391,356.862940,358.999825,209.730660,333.227604,212.780845,341.559351,343.793752,122.978550
1,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,459.499241,286.945160,388.054887,193.238409,315.705841,419.375390,245.853248,464.407538,...,267.986913,348.514492,273.177822,221.686398,276.333295,348.945423,219.593627,216.220845,214.391406,266.704047
2,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,289.487410,285.519009,396.846670,186.351404,313.766217,371.262126,244.913861,470.215439,...,4.633939,352.081746,232.706826,234.065257,303.631607,348.217363,254.426393,347.173315,349.183821,343.030674
3,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,288.618273,268.979498,382.164063,173.585077,302.824455,414.413653,229.864511,459.539536,...,227.107254,352.654799,234.858961,231.754416,234.017605,347.966360,231.258714,346.965449,348.791818,231.301306
4,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,313.312155,300.632514,407.919343,201.106650,327.683822,401.319738,254.663514,480.387762,...,234.847301,354.146060,362.431497,362.528120,238.848604,350.278534,346.834737,348.730438,347.520903,344.869274


In [ ]:
new_body_type_dict = {
    0: "head_top",
    1: "upper_jaw",
    2: "upper_left_jaw",
    3: "upper_right_jaw",
    4: "lower_jaw",
    5: "lower_left_jaw",
    6: "lower_right_jaw",
    7: "mouth_end_left",
    8: "mouth_end_right",
    9: "throat_base",
    10: "throat_end",
    11: "left_nostril",
    12: "right_nostril",
    13: "snout_top",
    14: "nose_bridge",
    15: "tongue",
    16: "left_eye",
    17: "right_eye",
    18: "left_ear_base",
    19: "left_ear_tip",
    20: "right_ear_base",
    21: "right_ear_tip",
    22: "neck_base",
    23: "neck_end",
    24: "withers"
}


In [3]:
body_type_dict = {
    0: "head_top",
    1: "upper_jaw",
    2: "upper_left_jaw",
    3: "upper_right_jaw",
    4: "lower_jaw",
    5: "lower_left_jaw",
    6: "lower_right_jaw",
    7: "mouth_end_left",
    8: "mouth_end_right",
    9: "throat_base",
    10: "throat_end",
    11: "left_nostril",
    12: "right_nostril",
    13: "snout_top",
    14: "nose_bridge",
    15: "tongue",
    16: "left_eye",
    17: "right_eye",
    18: "left_ear_base",
    19: "left_ear_tip",
    20: "right_ear_base",
    21: "right_ear_tip",
    22: "left_horn_base",
    23: "right_horn_base",
    24: "neck_base",
    25: "neck_end",
    26: "withers",
    27: "tail_base",
    28: "tail_end",
    29: "top_left_knee",
    30: "top_left_hoof",
    31: "top_right_knee",
    32: "top_right_hoof",
    33: "bottom_left_knee",
    34: "bottom_left_hoof",
    35: "bottom_right_knee",
    36: "bottom_right_hoof"
}

df = pd.read_csv("./pose_train.csv", header=None)

# Rename columns
df = df.rename(columns={
    0: "file_path", 
    1: "class_type"
})

# Rename remaining columns using loop
for i, (key, value) in enumerate(body_type_dict.items()):
    df = df.rename(columns={
        2*i+2: f"{value}_x", 2*i+3: f"{value}_y"
    })

removed_landmarks = ["left_horn_base",
                     "right_horn_base",
                      "tail_base",
                      "tail_end",
                      "top_left_knee",
                      "top_left_hoof",
                      "top_right_knee",
                      "top_right_hoof",
                      "bottom_left_knee",
                      "bottom_left_hoof",
                      "bottom_right_knee",
                      "bottom_right_hoof"]

# Remove unwanted landmarks
df = df.drop(columns=[col for landmark in removed_landmarks 
                      for col in [f"{landmark}_x", f"{landmark}_y"]])

df.head()

,file_path,class_type,head_top_x,head_top_y,upper_jaw_x,upper_jaw_y,upper_left_jaw_x,upper_left_jaw_y,upper_right_jaw_x,upper_right_jaw_y,...,right_ear_base_x,right_ear_base_y,right_ear_tip_x,right_ear_tip_y,neck_base_x,neck_base_y,neck_end_x,neck_end_y,withers_x,withers_y
0,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,296.361400,293.094178,397.665245,206.603340,377.376373,432.355077,305.081375,468.451430,...,199.306151,294.315023,292.633286,286.924213,115.693075,113.392481,53.178429,60.459720,279.581957,107.219965
1,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,459.499241,286.945160,388.054887,193.238409,315.705841,419.375390,245.853248,464.407538,...,190.420638,257.693499,264.025473,246.944134,109.557899,102.514060,43.726589,43.450309,226.496684,103.957907
2,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,289.487410,285.519009,396.846670,186.351404,313.766217,371.262126,244.913861,470.215439,...,226.628647,297.570500,302.248813,286.882875,144.955528,135.887252,69.888384,3.902289,284.363713,136.137280
3,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,288.618273,268.979498,382.164063,173.585077,302.824455,414.413653,229.864511,459.539536,...,192.475673,266.841775,268.900272,256.714435,107.872097,100.208299,38.425511,44.313932,227.684738,102.217632
4,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,313.312155,300.632514,407.919343,201.106650,327.683822,401.319738,254.663514,480.387762,...,200.583538,271.029247,278.318715,260.384214,117.659545,111.065335,50.618847,59.112247,261.044961,115.299045


## Drop Padding


In [4]:
# Generate new column names
# The first two columns are file_path and class
new_columns = ["file_path", "class"]
num_landmarks = (df.shape[1] - 2) // 2

for i in range(num_landmarks):
    new_columns.append(f"landmark_{i}_x")
    new_columns.append(f"landmark_{i}_y")
    # new_columns.append(f"landmark_{i}_confidence")

# Assign the new column names to the DataFrame
df.columns = new_columns

## Extract ground truth


In [5]:
def extract_salient(row):
    match = re.search(r"salient[12]", row[0])

    if match:
        salient_type = match.group()
    else:
        print("Salient type not found")

    if "salient1" == salient_type:
        return 0
    elif "salient2" == salient_type:
        return 1
    else:
        return -1


df["class"] = df.apply(extract_salient, axis=1)
df["class"] = df["salient"].astype(int)

df.head()

,file_path,class,landmark_0_x,landmark_0_y,landmark_1_x,landmark_1_y,landmark_2_x,landmark_2_y,landmark_3_x,landmark_3_y,...,landmark_20_y,landmark_21_x,landmark_21_y,landmark_22_x,landmark_22_y,landmark_23_x,landmark_23_y,landmark_24_x,landmark_24_y,salient
0,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,296.361400,293.094178,397.665245,206.603340,377.376373,432.355077,305.081375,468.451430,...,294.315023,292.633286,286.924213,115.693075,113.392481,53.178429,60.459720,279.581957,107.219965,0
1,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,459.499241,286.945160,388.054887,193.238409,315.705841,419.375390,245.853248,464.407538,...,257.693499,264.025473,246.944134,109.557899,102.514060,43.726589,43.450309,226.496684,103.957907,0
2,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,289.487410,285.519009,396.846670,186.351404,313.766217,371.262126,244.913861,470.215439,...,297.570500,302.248813,286.882875,144.955528,135.887252,69.888384,3.902289,284.363713,136.137280,0
3,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,288.618273,268.979498,382.164063,173.585077,302.824455,414.413653,229.864511,459.539536,...,266.841775,268.900272,256.714435,107.872097,100.208299,38.425511,44.313932,227.684738,102.217632,0
4,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,313.312155,300.632514,407.919343,201.106650,327.683822,401.319738,254.663514,480.387762,...,271.029247,278.318715,260.384214,117.659545,111.065335,50.618847,59.112247,261.044961,115.299045,0


## Extract angles between points

`angle_a_b_c` will be the angle for an angle abc.


In [6]:
# Function to calculate angle between three points (p0, p1, p2)
def calculate_angle(p0, p1, p2):
    v0 = np.array(p0) - np.array(p1)  # Vector from p1 to p0
    v2 = np.array(p2) - np.array(p1)  # Vector from p1 to p2
    cosine_angle = np.dot(v0, v2) / (np.linalg.norm(v0) * np.linalg.norm(v2))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Clip to handle precision errors
    return np.degrees(angle)

# Extract x and y coordinates from the dataframe
poses_x = df.iloc[:, 2::2].astype(float)
poses_y = df.iloc[:, 3::2].astype(float)

# Create a new dataframe to store angles
angles_df = pd.DataFrame()
angles_df['file_path'] = df['file_path']
angles_df['salient'] = df['salient']

# Calculate angles for each triplet of landmarks
num_landmarks = len(poses_x.columns) - 1

# Generate all unique triplets of landmark indices
landmark_indices = range(num_landmarks)
triplets = list(itertools.combinations(landmark_indices, 3))

# Calculate angles for all landmark triples
for count, triplet in enumerate(triplets):
    i, j, k = triplet
    p0 = (poses_x.iloc[:, i].values, poses_y.iloc[:, i].values)
    p1 = (poses_x.iloc[:, j].values, poses_y.iloc[:, j].values)
    p2 = (poses_x.iloc[:, k].values, poses_y.iloc[:, k].values)
    
    x_coords = [p0[0], p1[0], p2[0]]
    y_coords = [p0[1], p1[1], p2[1]]

    angles = [calculate_angle((x_coords[0][i], y_coords[0][i]), 
                            (x_coords[1][i], y_coords[1][i]), 
                            (x_coords[2][i], y_coords[2][i])) 
            for i in range(len(x_coords[0]))]
        
    # Create column names based on landmark indices
    col_name = f'angle_{i}_{j}_{k}'
    angles_df[col_name] = angles
    print(f"Completed {count} out of {len(triplets)}")

# The resulting dataframe will contain file_path, salient, and calculated angles
angles_df.head()

Completed 0 out of 2300
Completed 1 out of 2300
Completed 2 out of 2300
Completed 3 out of 2300
Completed 4 out of 2300
Completed 5 out of 2300
Completed 6 out of 2300
Completed 7 out of 2300
Completed 8 out of 2300
Completed 9 out of 2300
Completed 10 out of 2300
Completed 11 out of 2300
Completed 12 out of 2300
Completed 13 out of 2300
Completed 14 out of 2300
Completed 15 out of 2300
Completed 16 out of 2300
Completed 17 out of 2300
Completed 18 out of 2300
Completed 19 out of 2300
Completed 20 out of 2300
Completed 21 out of 2300
Completed 22 out of 2300
Completed 23 out of 2300
Completed 24 out of 2300
Completed 25 out of 2300
Completed 26 out of 2300
Completed 27 out of 2300
Completed 28 out of 2300
Completed 29 out of 2300
Completed 30 out of 2300
Completed 31 out of 2300
Completed 32 out of 2300
Completed 33 out of 2300
Completed 34 out of 2300
Completed 35 out of 2300
Completed 36 out of 2300
Completed 37 out of 2300
Completed 38 out of 2300
Completed 39 out of 2300
Completed 

/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = f

Completed 100 out of 2300
Completed 101 out of 2300
Completed 102 out of 2300


/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = f

Completed 103 out of 2300
Completed 104 out of 2300
Completed 105 out of 2300


...

Completed 2290 out of 2300
Completed 2291 out of 2300
Completed 2292 out of 2300
Completed 2293 out of 2300


/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = f

Completed 2294 out of 2300
Completed 2295 out of 2300
Completed 2296 out of 2300
Completed 2297 out of 2300
Completed 2298 out of 2300
Completed 2299 out of 2300


/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_745348/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = f

,file_path,salient,angle_0_1_2,angle_0_1_3,angle_0_1_4,angle_0_1_5,angle_0_1_6,angle_0_1_7,angle_0_1_8,angle_0_1_9,...,angle_20_21_22,angle_20_21_23,angle_20_21_24,angle_20_22_23,angle_20_22_24,angle_20_23_24,angle_21_22_23,angle_21_22_24,angle_21_23_24,angle_22_23_24
0,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,44.374562,30.037578,45.052788,73.332934,12.659892,18.180151,18.157994,30.677631,...,48.970740,47.930889,90.374036,155.059396,67.352954,46.330726,175.812655,46.599694,31.733441,28.585948
1,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,55.064070,64.995404,81.697327,13.045666,91.845234,56.107180,79.228773,51.135683,...,51.385459,51.038013,83.602434,159.422058,61.768951,37.282676,178.821775,42.369235,24.411620,23.580840
2,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,23.076981,19.114315,2.836898,65.319344,12.329033,16.014175,8.029015,46.279743,...,51.874240,58.654473,91.278318,177.171009,63.096949,30.253759,163.459019,43.726977,18.954017,28.714765
3,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,26.205250,16.399627,64.235546,65.737098,10.552773,25.233468,2.884013,30.332565,...,51.732622,50.211511,82.611431,155.746924,62.121271,38.294718,174.644891,43.223304,25.651542,21.817544
4,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,21.710153,14.792800,3.402954,60.316333,12.421984,5.349797,0.994694,16.051233,...,50.702384,49.272171,91.007927,155.175697,60.906875,39.764558,174.869025,41.213547,26.524588,22.823825


In [7]:
angles_df.to_csv("./angles.csv", index=False)

In [9]:
angles_df.head()

,file_path,salient,angle_0_1_2,angle_0_1_3,angle_0_1_4,angle_0_1_5,angle_0_1_6,angle_0_1_7,angle_0_1_8,angle_0_1_9,...,angle_20_21_22,angle_20_21_23,angle_20_21_24,angle_20_22_23,angle_20_22_24,angle_20_23_24,angle_21_22_23,angle_21_22_24,angle_21_23_24,angle_22_23_24
0,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,44.374562,30.037578,45.052788,73.332934,12.659892,18.180151,18.157994,30.677631,...,48.970740,47.930889,90.374036,155.059396,67.352954,46.330726,175.812655,46.599694,31.733441,28.585948
1,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,55.064070,64.995404,81.697327,13.045666,91.845234,56.107180,79.228773,51.135683,...,51.385459,51.038013,83.602434,159.422058,61.768951,37.282676,178.821775,42.369235,24.411620,23.580840
2,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,23.076981,19.114315,2.836898,65.319344,12.329033,16.014175,8.029015,46.279743,...,51.874240,58.654473,91.278318,177.171009,63.096949,30.253759,163.459019,43.726977,18.954017,28.714765
3,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,26.205250,16.399627,64.235546,65.737098,10.552773,25.233468,2.884013,30.332565,...,51.732622,50.211511,82.611431,155.746924,62.121271,38.294718,174.644891,43.223304,25.651542,21.817544
4,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,21.710153,14.792800,3.402954,60.316333,12.421984,5.349797,0.994694,16.051233,...,50.702384,49.272171,91.007927,155.175697,60.906875,39.764558,174.869025,41.213547,26.524588,22.823825
